In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

<h1>Loading database</h1>

In [4]:
from pypots.benchpots.datasets import preprocess_physionet2012
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1)

2024-10-29 21:38:29 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-10-29 21:38:29 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-10-29 21:38:29 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-10-29 21:38:30 [INFO]: Loaded successfully!


<h1>Training data</h1>

In [5]:
train_X = physionet2012_dataset['train_X']

In [ ]:
distribution_gender = train_X['Gender'].value_counts()
distribution_gender # 0: female, or 1: male

Gender
 1.0    4257
 0.0    3404
-1.0      10
Name: count, dtype: int64

In [11]:
distribution_ICUType = train_X['ICUType'].value_counts()
distribution_ICUType

ICUType
3.0    2750
4.0    2173
2.0    1617
1.0    1131
Name: count, dtype: int64

<h1>Validation data</h1>

In [15]:
validation_X = physionet2012_dataset['val_X']

<h1>Test data</h1>

In [16]:
test_X = physionet2012_dataset['test_X']